<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Часть 2
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План 2 части домашнего задания
    2.1. Введение
    2.2. Описание данных
    2.3. Предобработка данных
    2.4. Обучение и проверка моделей
    2.5. Заключение

### 2.1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://goo.gl/forms/z8zENbMiaEAeB7nG3) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в README [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2.2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://yadi.sk/d/krikdUic3Ggjyf) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 ../data/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l ../data/stackoverflow.10kk.tsv

10000000 ../data/stackoverflow.10kk.tsv
CPU times: user 2.77 s, sys: 664 ms, total: 3.44 s
Wall time: 1min 10s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 2.3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [3]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. Как видите, 10 Гб у меня обработались примерно за полторы минуты.

In [4]:
!python preprocess.py ../data/stackoverflow.10kk.tsv stackoverflow.vw

10000000it [01:17, 129051.02it/s]
Количество записей:  4389054
Должно быть       :  4389054


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [13]:
''' ВАШ КОД ЗДЕСЬ '''
!split --lines=1463018 stackoverflow.vw

In [16]:
!head -1463018 xaa > stackoverflow_train.vw

In [18]:
!head -1463018 xab > stackoverflow_valid.vw

In [19]:
!head -1463018 xac > stackoverflow_test.vw

In [25]:
!cut --d '|' -f1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt

In [26]:
!cut --d '|' -f1 stackoverflow_test.vw > stackoverflow_test_labels.txt

### 2.4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: bit_precision=28 и seed=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [27]:
import os
from sklearn.metrics import accuracy_score
with open('stackoverflow_valid_labels.txt') as true_file:
    true_labels = [int(label) for label in true_file.readlines()]

In [31]:
%%time
''' ВАШ КОД ЗДЕСЬ '''
results = {}
for pas in [1,3,5]:
    for ngram in [1,2,3]:
        print(f"pass: {pas}, ngram: {ngram}")
        vw_params_train = f"--oaa 10 -d stackoverflow_train.vw -f model_{pas}_{ngram}.vw --loss_function=hinge --passes {pas} -c --ngram {ngram} -b 28 --random_seed 17"
        vw_params_test = f"-i model_{pas}_{ngram}.vw -t -d stackoverflow_valid.vw -p pred_{pas}_{ngram}.txt"
    
        vw_train = os.system("vw " + vw_params_train)
        vw_test = os.system("vw " + vw_params_test)
        
        with open(f"pred_{pas}_{ngram}.txt") as pred_file:
            valid_prediction = [int(label) for label in pred_file.readlines()]
            
        key = f"{pas}_{ngram}"
        results[key] = accuracy_score(valid_prediction, true_labels)

pass: 1, ngram: 1
pass: 1, ngram: 2
pass: 1, ngram: 3
pass: 3, ngram: 1
pass: 3, ngram: 2
pass: 3, ngram: 3
pass: 5, ngram: 1
pass: 5, ngram: 2
pass: 5, ngram: 3
CPU times: user 7.48 s, sys: 312 ms, total: 7.79 s
Wall time: 44min 7s


In [32]:
results

{'1_1': 0.91666267947489366,
 '1_2': 0.93235694981196404,
 '1_3': 0.92997010289688853,
 '3_1': 0.91800784405933489,
 '3_2': 0.93053742332630218,
 '3_3': 0.9272544835401888,
 '5_1': 0.91802698257984527,
 '5_2': 0.93123529580634001,
 '5_3': 0.92814647529968874}

In [35]:
max(results.values())

0.93235694981196404

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- **Биграммы и 1 проход по выборке**
- Униграммы и 1 проход по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [37]:
''' ВАШ КОД ЗДЕСЬ '''
!vw -i model_1_2.vw -t -d stackoverflow_test.vw -p pred_test.txt


with open('pred_test.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]

with open('stackoverflow_test_labels.txt') as true_file:
    true_labels_test = [int(label) for label in true_file.readlines()]

print(f"Accuracy: {accuracy_score(valid_prediction, true_labels_test)}")



Generating 2-grams for all namespaces.
only testing
predictions = pred_test.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.250000 0.500000            4            4.0        5        5      158
0.250000 0.250000            8            8.0        2        2      116
0.125000 0.000000           16           16.0        6        6       74
0.093750 0.062500           32           32.0        1        1      324
0.078125 0.062500           64           64.0        4        4       52
0.078125 0.078125          128          128.0        7        7      250
0.070312 0.062500          256          256.0    

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- **Результаты почти одинаковы – отличаются меньше чем на 0.5%**

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [38]:
''' ВАШ КОД ЗДЕСЬ '''
!cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_new_train.vw

In [39]:
%%time
!vw --oaa 10 -d stackoverflow_new_train.vw -f final_model.vw --loss_function=hinge --passes 1 -c --ngram 2 -b 28 --random_seed 17

Generating 2-grams for all namespaces.
final_regressor = final_model.vw
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
creating cache_file = stackoverflow_new_train.vw.cache
Reading datafile = stackoverflow_new_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      320
0.500000 1.000000            2            2.0        4        1      134
0.750000 1.000000            4            4.0        7        1      174
0.750000 0.750000            8            8.0        7        1      188
0.750000 0.750000           16           16.0        7        7      416
0.781250 0.812500           32           32.0        7        2      346
0.750000 0.718750           64           64.0        3        3      406
0.648438 0.546875          128          128.0        1        7       56
0.617188 0.5

In [41]:
!vw -i final_model.vw -t -d stackoverflow_test.vw -p pred_final_model_test.txt

Generating 2-grams for all namespaces.
only testing
predictions = pred_final_model_test.txt
Num weight bits = 28
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        9        9      370
0.000000 0.000000            2            2.0        1        1       62
0.000000 0.000000            4            4.0        5        5      158
0.000000 0.000000            8            8.0        2        2      116
0.000000 0.000000           16           16.0        6        6       74
0.031250 0.062500           32           32.0        1        1      324
0.046875 0.062500           64           64.0        4        4       52
0.046875 0.046875          128          128.0        7        7      250
0.058594 0.070312          256       

In [44]:
with open('pred_final_model_test.txt') as pred_file:
    valid_prediction = [int(label) for label in pred_file.readlines()]

print(f"Accuracy: {accuracy_score(valid_prediction, true_labels_test)}")

Accuracy: 0.9364471250524601


In [46]:
(accuracy_score(valid_prediction, true_labels_test) - 0.9325346646452743) / (0.9325346646452743 * 0.01)

0.41955120335113671

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - **0.4%**
 - 0.8%
 - 1.2%

### 2.5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)